# Тестируем типовые сценарии и баги текстового чата

#### Для начала установите P2P соединение с телефоном

In [38]:
import asyncio
import datetime
from typing import Optional

import sirius_sdk
import ipywidgets as widgets
from IPython.display import display, Image, HTML

################### P2P connection ###################
P2P: Optional[sirius_sdk.Pairwise] = None
LABEL: str = 'Test-' + str(datetime.datetime.utcnow())
if LABEL.split(' ') > 1:
    # проще использовать только время без даты
    LABEL = LABEL[-1]
#######################################################

# Инициализируем подключение к Wallet в облаке
CFG = sirius_sdk.Config().setup_cloud(
    server_uri='https://agents.socialsirius.com',
    credentials='s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+rP2GFW7NHGJkINyK9gPhqmw2zGmFReH7De9JpfPW5ufQ=',
    p2p={
        "their_verkey": "6Ym63c9YhtJsCP1itowQozvkCAxATgeNCeJdGThvLvGP",
        "my_keys": [
          "5fEvX3eveEs6tuXeW8AqafDG1DLNUH8EpSWG49wVSScs",
          "2ooEHPR9qU3uo6H9KbV3VNuAChzEQG3NzkodBC59NR12tkmFBH3B7Htsdbe6FYxStdRQ52EamCg2Ux75EZeC7KyM"
        ]
   }
)


async def establish_connection():
    global P2P
    try:
        async with sirius_sdk.context(CFG):
            did, verkey = await sirius_sdk.DID.create_and_store_my_did()
            output.append_stdout(f'\n My Label: {LABEL} \n My DID: {did}\n My Key: {verkey}')
            
            manager = sirius_sdk.recipes.InvitationManager(
                me=sirius_sdk.Pairwise.Me(did, verkey),
                my_label=LABEL
            )
            invitation = await manager.make_invitation()
            invitation_url = 'https://example.com/invitation' + invitation.invitation_url
            
            output.append_display_data(HTML("<br/><span style='color:blue;'> Invitation string </span>"))
            output.append_stdout('_'*64 + '\n')
            output.append_stdout(invitation_url)
            output.append_stdout('\n' + '_'*64)
            
            output.append_stdout('\n Waiting QRCode...')
            qr_url = await sirius_sdk.generate_qr_code(invitation_url)
            image_qr = Image(url=qr_url, width = 250, unconfined=True)
            output.append_display_data(image_qr)
            
            # Start to listen...
            output.append_stdout('\nWait for connection requests ...\n')
            P2P = await manager.wait_connection(invitation)
            output.append_stdout('_'*64 + '\n')
            output.append_stdout(f'\n!!! Connection with "{P2P.their.label}" was established !!!!')
            output.append_stdout('\n' + '_'*64)
    except Exception as e:
        output.append_stderr(repr(e))
        
        
output = widgets.Output(layout={"width": "1000px"})
display(output)
asyncio.ensure_future(establish_connection())


Output(layout=Layout(width='1000px'))

<Task pending name='Task-98' coro=<establish_connection() running at /tmp/ipykernel_167/3446796242.py:28>>

# Тестируем *"Чат. Сообщение не доставляется при быстрой отправке сообщений"*
YouTrack: https://indilynx.youtrack.cloud/issue/SI-104/Chat-Soobshchenie-ne-dostavlyaetsya-pri-bystroi-otpravke-soobshchenii

Алгоритм тестирования:
  1. Задается количество сообщений для отправки
  2. Задается задержка (мсек) между отправками
  3. Отправляется пачкой
  
 

In [78]:
assert P2P, 'Установите соединение с мобилкой на предыдущих шагах!!!'

import asyncio
import datetime

GROUP_COUNTER = 1

input_msg_count = widgets.IntText(value='10', description='Сообщений:')
input_msg_delay = widgets.IntText(value='100', description='МСЕК:')
button_run = widgets.Button(description="!!! ОГОНЬ !!!", button_style='danger')
output = widgets.Output(layout={"width": "1000px"})


# Запускаем тест
async def run_test(msg_count: int, msg_delay_msec: int, ):
    global P2P
    global GROUP_COUNTER
    output.append_stdout(f'\nChat Label: "{LABEL}"')
    output.append_stdout(f'\nDetails: msg_count = {msg_count}; msg_delay_msec = {msg_delay_msec}')
    try:
        
        test_group_prefix = f'Group-{GROUP_COUNTER}/Test-'
        GROUP_COUNTER += 1
        msg_delay_sec = msg_delay_msec/1000
        
        async with sirius_sdk.context(CFG):
            for n in range(msg_count):
                
                msg_text = test_group_prefix + str(n+1)
                
                msg = sirius_sdk.aries_rfc.BasicMessage(
                    content=msg_text,
                    locale='ru',
                )
                out_time = str(datetime.datetime.utcnow()).replace('T', ' ') + 'Z'
                msg['~timing'] = {
                    "out_time": out_time,
                }
                ##### Отправка
                await sirius_sdk.send_to(msg, P2P)
                #########
                await asyncio.sleep(msg_delay_sec)
                
    except Exception as e:
        output.append_stderr(repr(e))


def on_run_clicked(b):
    output.clear_output()
    asyncio.ensure_future(run_test(msg_count=input_msg_count.value, msg_delay_msec=input_msg_delay.value))
    
button_run.on_click(on_run_clicked)
        
display(input_msg_count, input_msg_delay, button_run, output)

IntText(value=10, description='Сообщений:')

IntText(value=100, description='МСЕК:')

Button(button_style='danger', description='!!! ОГОНЬ !!!', style=ButtonStyle())

Output(layout=Layout(width='1000px'))